In [1]:
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm
from mpl_toolkits import mplot3d
import time
from klepto.archives import dir_archive

In [2]:
from rawFLIR_polarization_data_extraction import *
from slope2height import *

In [3]:
NUM_IMAGES=1
BEDFORMS = ['corals', 'canopy', 'rocks', 'dunes']
flow_speeds = ['fast', 'med', 'slow']
submergences = ['Deep', 'Intermed', 'Shallow']

# choose the data
bed_idx = 1
flow_idx = 0
submergence_idx = 2
TEST = 2

# SET the data variables
FLOW_SPEED = flow_speeds[flow_idx]
SUBMERGENCE = submergences[submergence_idx]
BEDFORM = BEDFORMS[bed_idx]

# source data folders
file_location = "../FLIR_Camera/{}_{}Flow_{}H_test{}_centeredCam_flume_LookAngle_35Deg".format(BEDFORM, FLOW_SPEED, SUBMERGENCE, TEST)
ref_image_name = 'reference.tiff'
save_location =  "../FLIR_Camera"
dark_files_location = "../FLIR_Camera/dark_imgs"
flat_files_location = "../FLIR_Camera/flat_imgs"
klepto_saved_data_origin = 'klepto_bulk_data_saves' 

In [4]:
flat_field_correction_params = flat_field_params(dark_files_location, flat_files_location)

In [5]:
im_theta1, im_phi1 = theta_phi(ref_image_name, file_location,
                               material = 'water',
                               flat_field_correct = 1,
                               gaussian_smoothing_sigma = 0,
                               num_images= NUM_IMAGES, correction_angle=0,
                               flat_field_correction_params = flat_field_correction_params)

Processing data for material : WATER
image_data shape is :  (2048, 2448)
im90 shape is :  (1024, 1224)
Stokes time:  0.0379786491394043 s
im_stokes0 shape is :  (1024, 1224)
DoLP time:  0.05097007751464844 s
im_DOLP shape is :  (1024, 1224)


C:\Users\tracy\Downloads\saksham_polarimetric_cam\scripts\rawFLIR_polarization_data_extraction.py:225: RuntimeWarning: invalid value encountered in true_divide
  alpha = 0.5*(np.tan(thI-thT)/np.tan(thI+thT))**2
C:\Users\tracy\Downloads\saksham_polarimetric_cam\scripts\rawFLIR_polarization_data_extraction.py:226: RuntimeWarning: invalid value encountered in true_divide
  eta = 0.5*(np.sin(thI-thT)/np.sin(thI+thT))**2
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:689: RuntimeWarning: invalid value encountered in greater
  above_bounds = x_new > self.x[-1]


Time for DOLP2Theta conversion is :  24.65065598487854 seconds
False
AoLP time:  0.05496859550476074 s


# SAVE the height, slope, raw data maps

In [6]:
filenames = os.listdir(file_location)
filenames.remove(ref_image_name) 
NUM_FRAMES = len(filenames)
print(NUM_FRAMES)

900


In [ ]:
for k in tqdm(range(4)):
    # set the frames to be processed and save locations
    SUBFOLDER = k+1
    data_foldername = "{}_{}Flow_{}H_test{}_subFolder{}".format(BEDFORM, FLOW_SPEED, SUBMERGENCE, TEST, SUBFOLDER)
    files = filenames[k*NUM_FRAMES//4: (k+1)*NUM_FRAMES//4]

    all_height_maps = np.zeros((im_theta1.shape[0],im_theta1.shape[1],len(files)))
    all_sx_maps = np.zeros((im_theta1.shape[0],im_theta1.shape[1],len(files)))
    all_sy_maps = np.zeros((im_theta1.shape[0],im_theta1.shape[1],len(files)))
    all_raw_frames_0filter = np.zeros((im_theta1.shape[0],im_theta1.shape[1],len(files)))

    i=-1
    for filename2 in tqdm(files): #
        i+=1
        im_theta2, im_phi2 = theta_phi(filename2, file_location,
                                   material = 'water',
                                   flat_field_correct = 1,
                                   gaussian_smoothing_sigma = 0,
                                   num_images= NUM_IMAGES, correction_angle=0,
                                   flat_field_correction_params = flat_field_correction_params)

        raw_img2 = cv2.imread(os.path.join(file_location, filename2))

        # get slopes and height maps
        s_x = np.tan(np.radians((im_theta2 - im_theta1)))
        s_y = np.tan(np.radians((im_phi2 - im_phi1)))
        s_hat_x, s_hat_y = slopeFieldFFT(s_x, s_y, suppress_fig = 1)
        height_map = slope2height(s_hat_x, s_hat_y)
        all_height_maps[:,:,i] = height_map
        all_sx_maps[:,:,i] = s_x
        all_sy_maps[:,:,i] = s_y
        all_raw_frames_0filter[:,:,i] = raw_img2[::2,::2,0]

    del im_theta2, s_x, s_y, s_hat_x, s_hat_y, height_map, raw_img2, im_phi2
    
    # Collate the data into a dict
    all_data_dict = {'all_height_maps': all_height_maps, 'all_sx_maps': all_sx_maps, \
                     'all_sy_maps':all_sy_maps, 'all_raw_frames_0filter':all_raw_frames_0filter}

    # Save using Klepto -- not pickle : directly to disk -- not memory intensive
    tic = time.time()
    demo = dir_archive(os.path.join(save_location, klepto_saved_data_origin, data_foldername), {}, \
                       serialized=True, cached=False)
    demo['all_height_maps'] = all_height_maps
    demo['all_sx_maps'] = all_sx_maps
    demo['all_sy_maps'] = all_sy_maps
    demo['all_raw_frames_0filter'] = all_raw_frames_0filter
    toc = time.time()
    print('time taken = ', toc - tic)
    demo.dump()
    
    # Empty some space
    del all_data_dict, all_height_maps, all_sx_maps, all_sy_maps, all_raw_frames_0filter, files, demo

  0%|                                                                                          | 0/225 [00:00<?, ?it/s]

Processing data for material : WATER
image_data shape is :  (2048, 2448)
im90 shape is :  (1024, 1224)
Stokes time:  0.04748392105102539 s
im_stokes0 shape is :  (1024, 1224)
DoLP time:  0.04697251319885254 s
im_DOLP shape is :  (1024, 1224)
Time for DOLP2Theta conversion is :  24.546147108078003 seconds
False
AoLP time:  0.047971487045288086 s


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: ComplexWarning: Casting complex values to real discards the imaginary part
  0%|▎                                                                               | 1/225 [00:28<1:45:05, 28.15s/it]

Processing data for material : WATER
image_data shape is :  (2048, 2448)
im90 shape is :  (1024, 1224)
Stokes time:  0.05496978759765625 s
im_stokes0 shape is :  (1024, 1224)
DoLP time:  0.06396126747131348 s
im_DOLP shape is :  (1024, 1224)
Time for DOLP2Theta conversion is :  25.420754432678223 seconds
False
AoLP time:  0.05151653289794922 s


  1%|▋                                                                               | 2/225 [00:54<1:43:07, 27.75s/it]

...

Processing data for material : WATER
image_data shape is :  (2048, 2448)
im90 shape is :  (1024, 1224)
Stokes time:  0.0259854793548584 s
im_stokes0 shape is :  (1024, 1224)
DoLP time:  0.031980276107788086 s
im_DOLP shape is :  (1024, 1224)
Time for DOLP2Theta conversion is :  22.01331329345703 seconds
False
AoLP time:  0.046971797943115234 s


 98%|████████████████████████████████████████████████████████████████████████████▌ | 221/225 [1:22:49<01:31, 22.76s/it]

Processing data for material : WATER
image_data shape is :  (2048, 2448)
im90 shape is :  (1024, 1224)
Stokes time:  0.02898430824279785 s
im_stokes0 shape is :  (1024, 1224)
DoLP time:  0.03897738456726074 s
im_DOLP shape is :  (1024, 1224)
Time for DOLP2Theta conversion is :  22.211199522018433 seconds
False
AoLP time:  0.04397392272949219 s


 99%|████████████████████████████████████████████████████████████████████████████▉ | 222/225 [1:23:12<01:08, 22.80s/it]

Processing data for material : WATER
image_data shape is :  (2048, 2448)
im90 shape is :  (1024, 1224)
Stokes time:  0.025985002517700195 s
im_stokes0 shape is :  (1024, 1224)
DoLP time:  0.03697776794433594 s
im_DOLP shape is :  (1024, 1224)
